## Шкарбаненко Михаил, Б05-907

## Задача 2.1

Выборка:

Рассматривается задача тестирования вакцины от некоторого вируса. Производство вакцины очень дорогое и затратное по времени, поэтому в день может быть произведена только одна ампула.

Требуется проверить, что вакцина помогает (вероятность заразиться меньше у человека с вакциной чем у человека без вакцины).

Эксперимент ставится следующим образом: каждый день есть два идентичных по здоровью человека. Один из людей принимает вакцину, а второй нет, после чего обоих ставят в одну среду с вирусом. В конце для проверяют кто заразился. (В таблице: s --- sick; h --- healthy)

Весь мир ждет вакцину от данного вируса, поэтому к руководству института постоянно приходят запросы о сроках завершения тестирования образца. Руководство поручило Вам оценить среднее время, которое понадобится на тестирования данной вакцины. А также провести анализ полученных данных на уровне значимости \alpha=0.05 и при ошибке второго рода beta=0.2.

Требуется:

Записать задачу формально;
Выполнить оценку среднего количества дней для принятия решения (учесть что истинная вероятность заразиться с вакциной и без равны p_1 = 0.2, p_2 = 0.5 соответственно);
Выполнить анализ данных и выяснить работает ли вакцина или нет.
Все выкладки должны быть сделаны аналитически, без использования компьютера.

## Решение

### 1. Подготовительная часть

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/andriygav/PSAD/master/labs/lab2/data/2.1.csv')
print(len(data))

30


In [ ]:
data.head()

,with vaccine,without vaccine
0,h,h
1,h,s
2,h,s
3,h,h
4,h,h


In [ ]:
data.loc[data["with vaccine"] == "h", "with vaccine"] = 1
data.loc[data["with vaccine"] == "s", "with vaccine"] = 0
data.loc[data["without vaccine"] == "h", "without vaccine"] = 1
data.loc[data["without vaccine"] == "s", "without vaccine"] = 0
data.head()

,with vaccine,without vaccine
0,1,1
1,1,0
2,1,0
3,1,1
4,1,1


### 2. Формальная постановка задачи и метод решения.

Воспользуемся теорией последовательного анализа:

$k_1 = \frac{p_1}{1-p_1}$, $k_2 = \frac{p_2}{1-p_2}$ - риски

$u = \frac{k_1}{k_2}$ - относительный риск

▶ $u = 1$ ⇔ p1 = p2,
▶ $u > 1$ ⇔ p1 > p2
▶ $u < 1 ⇔ p1 < p2$

Фиксируем «коридор» отклонений $u$ от 1, которые можно считать незначимыми:

[$u_L$, $u_U$] = [0.9, 1.1]

Гипотезы:

* $H_0: u \leq u_L$

* $H_1: u \geq u_U$

Статистика:

$d_m(X^m_1, X^m_2) = \sum\limits_{i=1}^{m}(1 - X_{1i})X_{2i}$

Пороги для принятия и отклонения нулевой гипотезы:

$a_m = \frac{lnB + mln\frac{1+u_U}{1+u_L}}{lnu_U - lnu_L}$

$r_m = \frac{lnA + mln\frac{1+u_U}{1+u_L}}{lnu_U - lnu_L}$ 

При каждом значении m:

▶ $d_m ⩾ r_m$ ⇒ отвергаем $H_0$, $u \geq u_U$ ;

▶ $d_m ⩽ a_m$ ⇒ принимаем $H_0$, $u \leq u_L$;

▶ $a_m < dm < r_m$ ⇒ процесс продолжается, добавляем элемент выборки.

Среднее число дней, необходимое для принятия решения, равно матожиданию размера выборки и может быть найдено по формуле:

$\mathbb{E}_u(n) = \frac{1}{p_1(1 - p_2) + p_2(1-p_1)} \frac{L(u)lnB + (1 - L(u))lnA}{\frac{u}{u+1}ln\frac{u_U(1 + u_L)}{u_L(1+u_U)} + \frac{1}{u+1}ln\frac{1+u_L}{1+u_U}}$

где $L(u) = \frac{A^h - 1}{A^h - B^h}$.

Параметр $h$ определяется из уравнения:

$\frac{u}{u+1} = \frac{1 - (\frac{1+u_L}{1+u_U})^h}{(\frac{u_U(1 + u_L)}{u_L(1+u_U)})^h-(\frac{1+u_L}{1+u_U})^h}$

Константы $A$ и $B$ определяются соотношениями из теории Вальда:

$A = \frac{1 - β}{α}$ $B = \frac{β}{1-\alpha}$





### 3. Численные расчеты

#### 3.1 Оценка количества дней для принятия решения 

In [ ]:
from scipy.optimize import fsolve
import math

In [ ]:
A = 16
B = 4 / 19
p1, p2 = 0.2, 0.5
u = 0.25
uL, uU = 0.9, 1.1

In [ ]:
def func(h, u=u, uL=uL, uU=uU):
  return (1 - ((1 + uL) / (1 + uU)) ** h) / (((uU * (1 + uL)) / (uL * (1 + uU))) ** h - ((1 + uL) / (1 + uU)) ** h) - u / (u + 1)

def L(h, A=A, B=B):
  return (A ** h - 1) / (A ** h - B ** h)

def E(h, u=u, uL=uL, uU=uU, A=A, B=B, p1=p1, p2=p2):
  ln1 = np.log((uU * (1 + uL)) / (uL * (1 + uU)))
  ln2 = np.log((1 + uL) / (1 + uU))
  return (1 / (p1 * (1 - p2) + p2 * (1 - p1))) * ((L(h) * np.log(B) + np.log(A) * (1 - L(h))) / ((u / (u + 1)) * ln1 + ln2 / (u + 1)))

In [ ]:
h = fsolve(func, 1)[0]
avg_num_days = math.ceil(E(h))
print(avg_num_days)

52


#### 3.2 Проверка эффективности вакцины

In [ ]:
def dm(x1, x2):
  return (1 - x1) @ x2

def am(m, B=B, uL=uL, uU=uU):
  return (np.log(B) + m * np.log((1 + uU) / (1 +uL))) / (np.log(uU) - np.log(uL))

def rm(m, A=A, uL=uL, uU=uU):
  return (np.log(A) + m * np.log((1 + uU) / (1 +uL))) / (np.log(uU) - np.log(uL))

In [ ]:
for m in range(1, data.shape[0]):
  x1 = data[0:m]['with vaccine'].values
  x2 = data[0:m]['without vaccine'].values
  d = dm(x1, x2)
  if d <= am(m):
    print("Размер выборки {}".format(m))
    print("dm <= am  --->  Принимаем гипотезу H0")
    break
  elif d >= rm(m):
    print("Размер выборки {}".format(m))
    print("dm >= rm  --->  Отвергаем гипотезу H0")
    break

Размер выборки 16
dm <= am  --->  Принимаем гипотезу H0


## Итоги

* Нулевая гипотеза о том, что для вакцинированного человека вероятность заразиться ниже, была принята. Для принятия решения понадобилась выборка из 16 наблюдений

* Среднее количество дней для принятия решения согласно оценке при учете истинных значений p1 и p2 равно 52